# Reproduce Published Results
This example shows how to use alternative software with the published data to reproduce published XAS results.

The input files to be used are the same as those of the Demeter (perl) example.

This example replicates one by one the same operations implementer in the Demeter example

# Libraries 
The following cell includes the libraries imported for  reproduction of results.

In [1]:
# Larch Libraries
# library to read ascii files
from larch.io import read_ascii
# libraries to handle data groups
from larch.utils import group2dict, dict2group
# library to normalise data
from larch.xafs import pre_edge
# import the larch.io libraries for managing athena files
from larch.io import create_athena, read_athena, extract_athenagroup

# File handling
from pathlib import Path

#plotting library
%matplotlib inline
import matplotlib.pyplot as plt

#library for writing to log
import logging

# custom libraries
# read/write csv data
import lib.handle_csv as hcsv

# Functions
The following cell includes the custom functions defined for enabling reproduction of results.

In [2]:


# initialise log file
def set_logger(log_file):
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=log_file, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    # prevent matplotlib font manager from writing to log
    logging.getLogger('matplotlib.font_manager').disabled = True
    logger.setLevel(logging.DEBUG)

#reading all with the same extension files from a dir
def get_files_list(source_dir, f_pattern):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob(f_pattern)):
        i_counter += 1
        files_list.append(filepath)
    return files_list

# Rename columns 
def rename_cols(xafs_group):
    # energy
    engy = xafs_group.col1
    # mu
    mu_e = xafs_group.col2
    # get a dictionary from te group
    xafs_dict = group2dict(xafs_group)
    # add mu and energy to the dictionary
    xafs_dict['energy'] = engy
    xafs_dict['mu'] = mu_e
    xafs_group = dict2group(xafs_dict)
    return xafs_group

# show plot of normalised data
def plot_normalised(xafs_group):
        plt.plot(xafs_group.energy, xafs_group.pre_edge, 'g', label='pre-edge') # plot pre-edge in green
        plt.plot(xafs_group.energy, xafs_group.post_edge, 'r', label='post-edge')# plot post-edge in green
        plt.plot(xafs_group.energy, xafs_group.mu, 'b', label=xafs_group.filename) # plot mu in blue
        plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
        plt.xlabel('Energy (eV)') # label y graph
        plt.ylabel('x$\mu$(E)') # label y axis
        plt.title("pre-edge and post_edge fitting to $\mu$")
        plt.legend() # show legend



# Input parameters (variables)

The variables in the next cell are the processing values that indicate where to get the data from, the pattern of the files to process and the number of files to process. These can be changed to process different datasets.


In [3]:
base_prefix = "pub_037"
athena_groups_file = "pub_037_athena.csv"
operations_list_file = "pub_037_operations.csv"

In [20]:
# create the path for storing results
base_path = Path("./" , base_prefix)
Path(base_path).mkdir(parents=True, exist_ok=True)

# create log file (store processing steps)
log_file = Path("./",base_path,"process.log")
print(log_file)
# set path for log
set_logger(log_file)

logging.info("Started processing")
logging.info("Input variables:")
logging.info("\tprefis     = " + base_prefix)
logging.info("\treading from athena file = " + athena_groups_file)
logging.info("\treading from operations file = " + operations_list_file )


# get the files list (from csv file), no ID, no headers
files_list, _ = hcsv.get_csv_data(athena_groups_file, "", False)
files_list = list(files_list.values())
# get the operations list (from csv file), no ID, no headers
operations_list, _ = hcsv.get_csv_data(operations_list_file, "", False)
operations_list = list(operations_list.values())


[root] INFO : Started processing
[root] INFO : Input variables:
[root] INFO : 	prefis     = pub_037
[root] INFO : 	reading from athena file = pub_037_athena.csv
[root] INFO : 	reading from operations file = pub_037_operations.csv


pub_037\process.log


In [21]:
files_list

[['..\\psdi_data\\pub_037\\XAFS_prj\\SnO2 0.9 2.6-13.5 gbkg.prj',
  'SnO2 0.9',
  'SnO2'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn K-edge\\PtSn_OCO.prj',
  'PtSn_OCO rebinned',
  'air'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn K-edge\\PtSn_OCA.prj',
  'PtSn_OCA rebinned',
  'Ar'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn K-edge\\PtSn_OCH.prj',
  'PtSn_OCH rebinned',
  'H2'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn foil.prj', 'merge', 'Sn Foil'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn K-edge\\PtSn_OC.prj',
  'PtSn_OC_MERGE_CALIBRATE rebinned',
  'PtSn'],
 ['..\\psdi_data\\pub_037\\XAFS_prj\\Sn K-edge\\PtSn_OCH_H2.prj',
  'PtSn_OCH rebinned',
  'H2-H2']]

In [22]:
operations_list

[['1',
  '0|1|2|3|4',
  '1|0|1|0|0|0|0|0|0',
  'Show normalised E plot for first five groups',
  '20|60|-20|180',
  '',
  '0',
  '0'],
 ['2',
  '0|1|2|3|4',
  '1|0|1|1|0|0|0|0|0',
  'Plot first derivate for first five groups',
  '20|60|-15|7',
  '',
  '0',
  '0'],
 ['3',
  '3|0|4',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn foil LCF for H2',
  '20|60|-20|60',
  '',
  '0',
  '0'],
 ['3',
  '2|0|4',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn foil LCF for Ar',
  '20|60|-20|60',
  '',
  '0',
  '0'],
 ['3',
  '1|0|4',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn foil LCF for air',
  '20|60|-20|60',
  '',
  '0',
  '0'],
 ['3',
  '3|0|6',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn H2-H2 LCF for H2',
  '20|60|-20|60',
  '',
  '0',
  '0'],
 ['3',
  '2|0|6',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn H2-H2 LCF for Ar',
  '20|60|-20|60',
  '',
  '0',
  '0'],
 ['3',
  '1|0|6',
  '0|0|0|1|0|0|0|0|0',
  'Showing SnO2 and Sn H2-H2 LCF for air',
  '20|60|-20|60',
  '',
  '0',
  '0']]